In [68]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [69]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [70]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [71]:
df = df.drop(['record_id', 'idade_faixa'], axis=1)

In [ ]:
X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]

In [ ]:
from sklearn.preprocessing import StandardScaler
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

In [ ]:
from sklearn.cluster import KMeans

km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )

In [ ]:
km.fit( X_afetam_eficacia )

In [ ]:
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [ ]:
df[['idade_diagnostico','ultima_informacao_paciente', 'tempo_seguimento', 'grau_histologico', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local', 'primeiro_peso', 'eficacia_tratamento']]

In [ ]:
# fazer alguns graficos de que tinha ((idade avançada com primeiro peso grande e eficacia tratamento))

In [ ]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))
# 0 é tratamento bom 
# 1 é tratamento ruim
# isso mostra dois casos o de uma pessoa mais idosa de 80 anos de idade que saiu viva ou seja o tratamento foi bom 
#o outro mostra que não tem nenhum caso em que a pessoa mais idosa sobreviveu e o tratamento foi classificado como negativo

In [ ]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))
# 0 é tratamento bom 
# 1 é tratamento ruim
# isso mostra dois casos o de uma mulher nova com menos de 30 anos uma que morreu e a outra que não de maneira que ela morreu, então o tratamento
# bem classificado, ou seja ele esta classificando bem

In [ ]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [ ]:
df

In [ ]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [ ]:
df.columns

In [ ]:
colunas_selecionadas = list(df.columns)

correlacoes = []

for i in range(len(colunas_selecionadas)):
    coluna1 = colunas_selecionadas[i]
    coluna2 = 'tratamento'
    corr = df[coluna1].corr(df['tratamento'])
    chave = f'{coluna1} x {coluna2}'
    correlacoes.append({'colunas': chave, 'correlacao': corr})

correlacoes_ordenadas = sorted(correlacoes, key=lambda x: x['correlacao'], reverse=True)

correlacoes_ordenadas

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['tratamento'], axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
models

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np

from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier

classifiers = [AdaBoostClassifier(), 
               RandomForestClassifier(),
               ExtraTreesClassifier(),
               RidgeClassifierCV(), 
               LinearSVC(),
               LogisticRegression(), 
               SVC(), 
               LGBMClassifier(),
               LinearDiscriminantAnalysis(), 
               RidgeClassifier(), 
               CalibratedClassifierCV(), 
               XGBClassifier(),
               NearestCentroid(), 
               SGDClassifier(), 
               BaggingClassifier(),
               BernoulliNB(),
               PassiveAggressiveClassifier(), 
               Perceptron(),
               NuSVC(), 
               DecisionTreeClassifier(), 
               KNeighborsClassifier(),
               LabelSpreading(),
               LabelPropagation(), 
               ExtraTreeClassifier(),
               GaussianNB(),
               DummyClassifier(),
               QuadraticDiscriminantAnalysis()]

precisions = []
recalls = []

for classifier in classifiers:
    clf = classifier.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))

In [ ]:
models['Precision'] = 0
models['Recall'] = 0   

for i in range(0, len(precisions)):
  models['Precision'].iloc[i] = precisions[i]
  models['Recall'].iloc[i] = recalls[i]

models

In [ ]:
# logo usaremos o modelo AdaBoostClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()

params = {'n_estimators': [50, 100, 200],
          'learning_rate': [0.1, 0.5, 1.0]}

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test, y_test))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Obtenha as previsões do modelo e os rótulos verdadeiros
y_pred = grid_search.predict(X_test)
y_true = y_test

# Crie a matriz de confusão usando sklearn.metrics.confusion_matrix
cm = confusion_matrix(y_true, y_pred)

# Crie a figura da matriz de confusão usando seaborn.heatmap
sns.set(font_scale=1.4)
sns.heatmap(
    cm,
    annot=True,
    cmap='Blues',
    fmt='d',
    xticklabels=['Predicted 0', 'Predicted 1'],
    yticklabels=['True 0', 'True 1'],
    cbar=True
)

# Adicione o título e rótulos de eixo à figura
plt.title("Matriz de confusão")
plt.xlabel("Rótulos previstos")
plt.ylabel("Rótulos verdadeiros")

# Mostre a figura
plt.show()